# Fitting a Logistic Regression Model - Lab

## Introduction

In the last lesson you were given a broad overview of logistic regression. This included an introduction to two separate packages for creating logistic regression models. In this lab, you'll be investigating fitting logistic regressions with `statsmodels`. For your first foray into logistic regression, you are going to attempt to build a model that classifies whether an individual survived the [Titanic](https://www.kaggle.com/c/titanic/data) shipwreck or not (yes, it's a bit morbid).


## Objectives

In this lab you will: 

* Implement logistic regression with `statsmodels` 
* Interpret the statistical results associated with model parameters

## Import the data

Import the data stored in the file `'titanic.csv'` and print the first five rows of the DataFrame to check its contents. 

In [2]:
# Import the data
import pandas as pd
df = pd.read_csv('titanic.csv')

df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Define independent and target variables

Your target variable is in the column `'Survived'`. A `0` indicates that the passenger didn't survive the shipwreck. Print the total number of people who didn't survive the shipwreck. How many people survived?

In [3]:
# Total number of people who survived/didn't survive
import numpy as np
died = (df['Survived'] == 0).sum()
survived = df.shape[0]-died
print(f'{survived} people survived and {died} people died.')

342 people survived and 549 people died.


Only consider the columns specified in `relevant_columns` when building your model. The next step is to create dummy variables from categorical variables. Remember to drop the first level for each categorical column and make sure all the values are of type `float`: 

In [7]:
# Create dummy variables
relevant_columns = ['Pclass', 'Age', 'SibSp', 'Fare', 'Sex', 'Embarked', 'Survived']
dummy_dataframe = pd.get_dummies(df[relevant_columns], columns = ['Pclass','Sex','Embarked'], drop_first = True, dtype = float)

dummy_dataframe.shape

(891, 9)

Did you notice above that the DataFrame contains missing values? To keep things simple, simply delete all rows with missing values. 

> NOTE: You can use the [`.dropna()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html) method to do this. 

In [8]:
# Drop missing rows
dummy_dataframe = dummy_dataframe.dropna()
dummy_dataframe.shape

(714, 9)

Finally, assign the independent variables to `X` and the target variable to `y`: 

In [9]:
# Split the data into X and y
y = dummy_dataframe['Survived']
X = dummy_dataframe.drop('Survived', axis = 1)

## Fit the model

Now with everything in place, you can build a logistic regression model using `statsmodels` (make sure you create an intercept term as we showed in the previous lesson).  

> Warning: Did you receive an error of the form "LinAlgError: Singular matrix"? This means that `statsmodels` was unable to fit the model due to certain linear algebra computational problems. Specifically, the matrix was not invertible due to not being full rank. In other words, there was a lot of redundant, superfluous data. Try removing some features from the model and running it again.

In [11]:
# Build a logistic regression model using statsmodels
import statsmodels.api as sm
X = sm.add_constant(X)

logit_model = sm.Logit(y, X)
results = logit_model.fit()

Optimization terminated successfully.
         Current function value: 0.443266
         Iterations 6


## Analyze results

Generate the summary table for your model. Then, comment on the p-values associated with the various features you chose.

In [12]:
# Summary table
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:               Survived   No. Observations:                  714
Model:                          Logit   Df Residuals:                      705
Method:                           MLE   Df Model:                            8
Date:                Thu, 08 Jul 2021   Pseudo R-squ.:                  0.3437
Time:                        09:36:42   Log-Likelihood:                -316.49
converged:                       True   LL-Null:                       -482.26
Covariance Type:            nonrobust   LLR p-value:                 7.889e-67
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          4.4362      0.534      8.302      0.000       3.389       5.484
Age           -0.0431      0.008     -5.190      0.000      -0.059      -0.027
SibSp         -0.3804      0.125     -3.041      0.002      -0.626      -0.135
Fare           0.0012      0.002      0.466      0.642      -0.004       0.006
Pclass_2      -1.2075      0.328     -3.686      0.000      -1.850      -0.565
Pclass_3      -2.4221      0.340     -7.129      0.000      -3.088      -1.756
Sex_male      -2.6233      0.218    -12.035      0.000      -3.051      -2.196
Embarked_Q    -0.8259      0.598     -1.381      0.167      -1.998       0.347
Embarked_S    -0.4137      0.274     -1.511      0.131      -0.951       0.123
==============================================================================
"""

In [13]:
# Your comments here
# Based on the p-values, Fare and Embarked are not significant factors.

## Level up (Optional)

Create a new model, this time only using those features you determined were influential based on your analysis of the results above. How does this model perform?

In [15]:
# Your code here
relevant_columns = ['Pclass', 'Age', 'SibSp', 'Sex', 'Survived']
dummy_dataframe = pd.get_dummies(df[relevant_columns], columns = ['Pclass','Sex'], drop_first = True, dtype = float)

dummy_dataframe = dummy_dataframe.dropna()

y = dummy_dataframe['Survived']
X = dummy_dataframe.drop('Survived', axis = 1)

X = sm.add_constant(X)

logit_model = sm.Logit(y, X)
results = logit_model.fit()
results.summary()

Optimization terminated successfully.
         Current function value: 0.445774
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:               Survived   No. Observations:                  714
Model:                          Logit   Df Residuals:                      708
Method:                           MLE   Df Model:                            5
Date:                Thu, 08 Jul 2021   Pseudo R-squ.:                  0.3400
Time:                        09:39:04   Log-Likelihood:                -318.28
converged:                       True   LL-Null:                       -482.26
Covariance Type:            nonrobust   LLR p-value:                 9.745e-69
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          4.3342      0.451      9.617      0.000       3.451       5.218
Age           -0.0448      0.008     -5.442      0.000      -0.061      -0.029
SibSp         -0.3802      0.122     -3.129      0.002      -0.618      -0.142
Pclass_2      -1.4144      0.285     -4.967      0.000      -1.972      -0.856
Pclass_3      -2.6526      0.286     -9.280      0.000      -3.213      -2.092
Sex_male      -2.6277      0.215    -12.235      0.000      -3.049      -2.207
==============================================================================
"""

In [16]:
# Your comments here
# This model has a similar 'Pseudo R-squared' to the previous one (sligtly less high), and all
# of the features used are significant.

## Summary 

Well done! In this lab, you practiced using `statsmodels` to build a logistic regression model. You then interpreted the results, building upon your previous stats knowledge, similar to linear regression. Continue on to take a look at building logistic regression models in Scikit-learn!